# Building a Data Preprocessing Pipeline <div class="tocSkip">


## Hilfsfunktionen und Einstellungen <div class="tocSkip">


In [3]:
%run "../../settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# path to import blueprints packages
sys.path.append('../packages')




In [6]:
def txa_pd_read_sql(database, sql):
    """
    Open connection to sqlite database and return result 
    of sql query as pandas datafram
    :param sql: query
    :return: dataframe
    """
    with sqlite3.connect(database) as con:
        cur = con.cursor()
        df = pd.read_sql_query(sql, con, )
        return df
    
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

def filtering(liste):
    ergebnis= [t for t in liste if t not in stopwords]
    ergebnis = " ".join(ergebnis)
    return ergebnis

## Blueprint: Standardizing Attribute Names 

In [7]:
import pandas as pd
import textacy

In [12]:
df = txa_pd_read_sql('../../resources/bt_reden_final.sqlite','SELECT * FROM bt_reden_prepared')

df.head(1)

,speech_id,speaker,affiliation,text,session_id,session_date,session_start,session_end,legislative_period,clean_text,adjs_verbs,nouns,entities,noun_phrases,adj_noun_phrases,length
0,ID194300100,Olaf Scholz,Bundesminister BMF,Herr Präsident! Meine Damen und Herren! Griechenland wird das dritte Hilfsprogramm planmäßig am 20. August 2018 beenden. Damit kann Griechenland die Rettungsschirme nach acht Jahren verlassen und ...,43,2018-06-29,09:00,17:07,19,Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen Europäische Union Euro-Gruppe gemeinsam gelin...,20. beenden verlassen stehen anschließen europäisch gelingen helfen europäische herausgekommen stehen glauben zeigen bewältigen diskutieren sagen europäisch finden glauben vereinbarte vorstellen u...,Griechenland Hilfsprogramm August Griechenland Rettungsschirme Chance Bein Nachricht Nachricht Union Euro-Gruppe Hilfsmaßnahmen Portugal Irland Schwierigkeit Nation Ergebnis Solidarität Hilfsnotwe...,Griechenland/LOC Griechenland/LOC Europäischen_Union/ORG Euro-Gruppe/ORG Portugal/LOC Irland/LOC Europa/LOC Europa/LOC Europäischen_Union/ORG Europa/LOC Euro-Gruppe/ORG Deutsche_Bundestag/ORG Grie...,Chance_Bein Nachricht_Nachricht Solidarität_Hilfsnotwendigkeiten Stärke_Kooperation Herausforderung_Zukunft Position_Frage Grundlage_Herausforderung Umgang_Fluchtmigration Umgang_Fluchtmigration_E...,20._August europäisch_Union europäische_Nation europäisch_Union vereinbarte_Paket deutsch_Bundestag deutsch_Bundestag_Maßnahme unterhalten_Entwicklung geschehen_Falle klappen_Augenblick klappen_Au...,1286


# Cleaning text data
using regular expressions to identify and remove unwanted patterns in the data.

## Blueprint: Identify Noise with Regular Expressions 


In [10]:
import re

RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\^^$%@]')
text = df['text']
def impurity(text, min_len=10):
    """returns the share of suspicious characters in a text"""
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

print(f'About {round(impurity(text.all()) *100,2)}% of the characters are impure')


About 0% of the characters are impure


## Blueprint: Character Normalization with textacy


our data are written in german so some of the chars are accented. This could lead to problems while analysing, so we need to normelize our data. 

In [133]:
import textacy.preprocessing as tprep
def normalize(text):
    text = tprep.normalize_hyphenated_words(text)
    text = tprep.normalize_quotation_marks(text)
    text = tprep.normalize_unicode(text)
    text = tprep.remove_accents(text)
    return text

# Tokenization

## Linguistic Processing with spaCy


Liguistic provides an integrated pipeline of processing components, by default a tokenizer, a part-of-speech tagger, a dependency parser, and a named-entity recognizer <br>
This Language object (nlp.pipleline) contains the shared vocabulary, the model, and the processing pipeline. spaCy’s tokenizer is pretty fast, but all other steps are based on neural models and consume a significant amount of time. Compared to other libraries, though, spaCy’s models are among the fastest.
<br>
<br>
Often you will only need the tokenizer and the part-of-speech tagger. In this case, you should disable the parser like this:

In [135]:
import spacy
nlp = spacy.load("de_core_news_md",disable=['parser'])


## Processing Text


### Printing all tokens in doc, each token represents an object <div class="tocSkip">

## utility for generating a table containing the tokens <div class='tocSkip'>

In [136]:
def display_nlp(doc, include_punct=False):
    """Generate data frame for visualization of spaCy tokens."""
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_,
                   'is_stop': t.is_stop, 'is_alpha': t.is_alpha,
                   'pos_': t.pos_, 'dep_': t.dep_,
                   'ent_type_': t.ent_type_, 'ent_iob_': t.ent_iob_}
            rows.append(row)
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    return df

## Blueprint: Customizing Tokenization

In [137]:
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \
                       compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):

    # use default patterns except the ones matched by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes
                if pattern not in ['\(']]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes
                if pattern not in ['\)']]
    infixes  = [pattern for pattern in nlp.Defaults.infixes
                if not re.search(pattern, 'xx-xx')]

    return Tokenizer(vocab          = nlp.vocab,
                     rules          = nlp.Defaults.tokenizer_exceptions,
                     prefix_search  = compile_prefix_regex(prefixes).search,
                     suffix_search  = compile_suffix_regex(suffixes).search,
                     infix_finditer = compile_infix_regex(infixes).finditer,
                     token_match    = nlp.Defaults.token_match)



## Blueprint: Working with Stop Words


## Blueprint: Extracting Lemmas Based on Part of Speech


In [139]:
def extract_noun_phrases(doc, preceding_pos=['NOUN'], sep='_'):
    patterns = []
    for pos in preceding_pos:
        patterns.append(f"POS:{pos} POS:NOUN:+")
    spans = textacy.extract.matches(doc, patterns=patterns)
    return [sep.join([t.lemma_ for t in s]) for s in spans]

# Constructing the pipeline 

In [138]:
# def delete_stopwords(doc):
#     with open('resources/stopwords_de.txt', encoding='utf-8') as file:
#         stopwords = file.read().splitlines()
#     non_stop = [t for t in doc if not t.is_stop and not t.is_punct or (not t.is_stop and t.text not in stopwords and not t.is_punct)]
#     non_stop = [t for t in non_stop if not t.text.isnumeric()]
#     return non_stop

# textNoStopps = delete_stopwords(doc)
# len(textNoStopps)

<div><img src="../resources/firstPipeline.png" width="600"/></div>

<div><img src="../resources/second.png" width="600"/></div>

In [140]:
#our df
df.head(1)

,speech_id,speaker,affiliation,text,session_id,session_date,session_start,session_end,legislative_period,clean_text,adjs_verbs,nouns,entities,noun_phrases,adj_noun_phrases,full_text,lemmas
0,ID194300100,Olaf Scholz,Bundesminister BMF,Herr Präsident! Meine Damen und Herren! Griechenland wird das dritte Hilfsprogramm planmäßig am 20. August 2018 beenden. Damit kann Griechenland die Rettungsschirme nach acht Jahren verlassen und ...,43,29.06.2018,09:00,17:07,19,Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen Europäische Union Euro-Gruppe gemeinsam gelin...,20. beenden verlassen stehen anschließen europäisch gelingen helfen europäische herausgekommen stehen glauben zeigen bewältigen diskutieren sagen europäisch finden glauben vereinbarte vorstellen u...,Griechenland Hilfsprogramm August Griechenland Rettungsschirme Chance Bein Nachricht Nachricht Union Euro-Gruppe Hilfsmaßnahmen Portugal Irland Schwierigkeit Nation Ergebnis Solidarität Hilfsnotwe...,,Chance_Bein Nachricht_Nachricht Solidarität_Hilfsnotwendigkeiten Stärke_Kooperation Herausforderung_Zukunft Position_Frage Grundlage_Herausforderung Umgang_Fluchtmigration Umgang_Fluchtmigration_E...,20._August europäisch_Union europäische_Nation europäisch_Union vereinbarte_Paket deutsch_Bundestag deutsch_Bundestag_Maßnahme unterhalten_Entwicklung geschehen_Falle klappen_Augenblick klappen_Au...,Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen Europäische Union Euro-Gruppe gemeinsam gelin...,Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen europäisch Union Euro-Gruppe gemeinsam geling...


In [141]:
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import spacy
from spacy import displacy

def remove_stops_tokenize_and_extract(text,**kwargs):
    nlp.tokenizer = custom_tokenizer(nlp)
    docTokens = nlp(text)
    ergebnis =[t.lemma_ for t in textacy.extract.words(docTokens, **kwargs)]
    ergebnis = " ".join(ergebnis)
    return ergebnis

def extract_entities(doc, include_types=None, sep='_'):

    ents = textacy.extract.entities(doc, 
             include_types=include_types, 
             exclude_types=None, 
             drop_determiners=True, 
             min_freq=1)
    ents = [e for e in ents if len(e) <= 2]
    # append label and entity text like Angela_Merkel/PER
    ents = [sep.join([t.text for t in e]).strip('.\n')+'/'+e.label_ for e in ents]
    return ents



def extract_nlp(doc):
    nlp.tokenizer = custom_tokenizer(nlp)
    return {
    'lemmas'          : extract_lemmas(doc,exclude_pos = ['PART', 'PUNCT', 'DET', 'PRON', 'SYM', 'SPACE'],filter_stops = False),
    'adjs_verbs'      : extract_lemmas(doc, include_pos = ['ADJ', 'VERB']),
    'nouns'           : extract_lemmas(doc, include_pos = ['NOUN', 'PROPN']),
    'noun_phrases'    : extract_noun_phrases(doc, ['NOUN']),
    'adj_noun_phrases': extract_noun_phrases(doc, ['ADJ']),
    'entities'        : extract_entities(doc, ['PER', 'ORG', 'GPE', 'LOC'])
    }

tokens = textacy.extract.words(doc, 
            filter_stops = True,           # default True, no stopwords
            filter_punct = True,           # default True, no punctuation
            filter_nums = True,            # default False, no numbers
            include_pos = ['ADJ', 'NOUN'], # default None = include all
            exclude_pos = None,            # default None = exclude none
            min_freq = 1)                  # minimum frequency of words


def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]




def text_process(dfColumn):
    nlp = spacy.load("de_core_news_md",disable=['parser','ner'])
    ## clean
#     for i in tqdm(range(10000)):
#         textSerie = dfColumn.map(text_clean)
    ## remove stops and tokenizse
    textSerie = dfColumn.progress_apply(remove_stops_tokenize_and_extract)
    textSerie = textSerie.apply(str.split).progress_apply(filtering)
    df['clean_' + dfColumn.name] = textSerie

    
def comments_process(dfColumn):
    df['clean_' + dfColumn.name] = dfColumn.map(comment_clean)

def p_name_process(dfColumn):
    df['clean_' + dfColumn.name] = dfColumn.map(p_name_clean)

def tageso_process(dfColumn):
    df['clean_' + dfColumn.name] = dfColumn.map(tageso_clean)
    
def name_process(dfColumn):
    df['clean_' + dfColumn.name] = dfColumn.map(name_clean)

# pd.options.display.max_rows = 99999
# text_process(df['text'])
df.head(1)

speech_id      speaker         affiliation  \
0  ID194300100  Olaf Scholz  Bundesminister BMF   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [142]:
nlp = spacy.load('de_core_news_md', disable=['parser','ner']) # ggf. auch NER abschalten
df['full_text'] =  df['clean_text']
# +'.\n\n' \
#                  + df['clean_p_name'] +'.\n\n'  \
#                  + df['rede'] + '.\n\n' \
#                  + df['redner']  +'.\n\n' \
#                  + df['clean_name'] + '.\n\n'\
#                  + df['clean_kommentare']

df[['full_text']]

full_text
0      Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen Europäische Union Euro-Gruppe gemeinsam gelingen Hilfsmaßnahmen Portugal Irland finanziell Schwierigkeit helfen europäische Nation gewissermaßen Ergebnis europäisch Solidarität Hilfsnotwendigkeiten herausgekommen Bein stehen glauben zeigen Europa Stärke Kooperation gemeinsam Herausforderung Zukunft bewältigen Europa Kooperation diskutieren Grund ausdrücklich sagen Staatschefs Europäische Union gestern Nacht gemeinsam Position Frage Asyl- Migrationspolitik finden Fortschritt glauben Grundlage Herausforderung Europa Umgang Fluchtmigration Erfolg Euro-Gruppe vereinbarte Paket vorstellen umfasst breit Unterstützung ausdrücklich bitten deutschen Bundestag Maßnahme beantragen unterstützen freuen Debatte geschehen unterhalten Entwicklung Griechenland letzt zutragen genau hinschauen stellen fest Reform unternehmen miteinander feststellen Griechenland gut Weg Anstrengung Regierung Parlamentes Bürgerin Bürger Griechenland nehmen groß Hilfe leisten Europäer Zurverfügungstellung Fähigkeit Kredit aufnehmen weitergeben Manchmal verwechseln politisch Debatte Direktzahlungen Kredit verleihen Irrtum bleiben wiederholen tatsächlich geschehen Falle Irland Portugal klappen Augenblick Land Zugang Finanzmärkten verlieren sorgen Zugang Hilfe erhalten bleiben Kredit gemeinschaftlich aufnehmen Fall EFSF ESM zurückzahlen Selbstverständlich umfasst lang Zeitraum Deutschland Kredit schwierig Situation aufnehmen wenig zurückzahlen Ding langfristig hören Natur Sache glauben notwendig Schritt letzte Tranche 15 Milliarde Euro bewilligen eigentlich Botschaft 15 Milliarde Euro letzt Tranche keineswegs gesamt Kapazität ESM Griechenland Aussicht stellen ausschöpfen ermöglichen möglicherweise Anspruch nehmen Anspruch nehmen letzte Tranche dienen Übergang Rückkehr Refinanzierung Kapitalmärkte begleiten Genauso Portugal Irland genauso Griechenland funktionieren Klar lang Programm Hilfe groß ergänzen Entscheidung Zukunft wichtig Laufzeitverlängerung Zinsstundung Krediterleichterungen schaffen Aussicht stellen 2016. Bundesminister Scholz Ehrhorn AfD gerne Zwischenfrage stellen Gestatten Bitte Minister vorgestern Frage konfrontieren ausführen Bundestag August 2015 Rettungspaket zustimmen einzig wichtig Voraussetzung nämlich IWF Rettungspaket beteiligen Beteiligung geben rede explizit vorgestern Rettungspaket Wahrheit entsprechen vorgestern fragen wiederholen Frage Geschäftsgrundlage Zustimmung Bundestag erlöschen Auszahlung Rettungspaketes befürworten Dank schön Schöne Frage wiederholen gleich Frage stellen wiederholen gleich Antwort geben IWF dabeihaben IWF Kredit Höhe 10 Milliarde Euro laufen 2024 zurückzahlen Programm Rolle spielen IWF Geld kümmern vorstellen überlegen eigentlich Sinn obwohl ESM-Kreditrahmen ausschöpfen IWF 1,6 Milliarde obendrauf geben lassen gewissermaßen Situation wiederholen ausdrücklich IWF Nachprogrammüberwachung politische Zweck Beschluß Haus erreichen Übrigen Punkt kommen hinten Rede weglassen sagen verlängern Zahlungszeiträume helfen Kredittragfähigkeit sorgen Rückkehr Refinanzierung Kapitalmärkte gelingen eingeübt Praxis erreichen verzichten Zinsaufschlag Beschluß 2016 2017 ankündigen Gewinn Zentralbank anfangs gegeben bilateral Darlehen erhalten griechische Politik erreichbar all Ding Weg Zustimmung bitten hören vereinbart sprechen kommen Nachprogrammüberwachung geben Anstrengung Bürgerin Bürger Griechenland Regierung unternehmen aufhören vergeblich gerne Politik Griechenland Zukunft entwickeln entsprechen Reformmaßnahmen letzt orientieren Klar Spielräume groß umso groß groß wirtschaftlich Erfolg Land Fall Nachprogrammüberwachung sicherstellen Griechenland Zukunft Reformpolitik verfolgen letzt einschlagen Schluss sagen Zeichen europäisch Solidarität geben erreichen europäisch Union gemeinsam zeigen zusammenhalten Lage Problem lösen Problem Wirtschaft organisieren Staatsfin

In [143]:
nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())
print(nlp_columns)

for col in nlp_columns:
    df[col] = None

['lemmas', 'adjs_verbs', 'nouns', 'noun_phrases', 'adj_noun_phrases', 'entities']


In [150]:
batch_size = 100
num_batches = math.ceil(len(df) / batch_size)

for i in tqdm(range(0, len(df), batch_size), total=num_batches):
    
    # spaCy Batch-Verarbeitung mit nlp.pipe, liefert eine Liste von Ergebnis-Docs
    docs = nlp.pipe(df['clean_text'][i:i+batch_size])
    
    # Extraktion der Lemmas und Eintragen im DataFrame für einen Batch
    for j, doc in enumerate(docs):
        for col, values in extract_nlp(doc).items():
            df[col].iloc[i+j] = values
            
for column in nlp_columns:
    df[column] = df[column].progress_map(lambda tokens: " ".join(tokens))
    
df[nlp_columns].head(10)

100%|████████████████████████████████████████████████████████████████████████████████| 247/247 [19:44<00:00,  4.80s/it]


  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

  0%|          | 0/24647 [00:00<?, ?it/s]

lemmas  \
0  Griechenland Hilfsprogramm planmäßig 20. August 2018 beenden Griechenland Rettungsschirme verlassen Chance Bein stehen Nachricht Nachricht anschließen europäisch Union Euro-Gruppe gemeinsam gelingen Hilfsmaßnahmen Portugal Irland finanziell Schwierigkeit helfen europäische Nation gewissermaßen Ergebnis europäisch Solidarität Hilfsnotwendigkeiten herausgekommen Bein stehen glauben zeigen Europa Stärke Kooperation gemeinsam Herausforderung Zukunft bewältigen Europa Kooperation diskutieren Grund ausdrücklich sagen Staatschefs europäisch Union gestern Nacht gemeinsam Position Frage Asyl- Migrationspolitik finden Fortschritt glauben Grundlage Herausforderung Europa Umgang Fluchtmigration Erfolg Euro-Gruppe vereinbarte Paket vorstellen umfasst breit Unterstützung ausdrücklich bitten deutsch Bundestag Maßnahme beantragen unterstützen freuen Debatte geschehen unterhalten Entwicklung Griechenland letzt zutragen genau hinschauen stellen fest Reform unternehmen miteinander feststellen Griechenland gut Weg Anstrengung Regierung Parlamentes Bürgerin Bürger Griechenland nehmen groß Hilfe leisten Europäer Zurverfügungstellung Fähigkeit Kredit aufnehmen weitergeben Manchmal verwechseln politisch Debatte Direktzahlungen Kredit verleihen Irrtum bleiben wiederholen tatsächlich geschehen Falle Irland Portugal klappen Augenblick Land Zugang Finanzmärkten verlieren sorgen Zugang Hilfe erhalten bleiben Kredit gemeinschaftlich aufnehmen Fall EFSF ESM zurückzahlen Selbstverständlich umfasst langen Zeitraum Deutschland Kredit schwierig Situation aufnehmen zurückzahlen Ding langfristig hören Natur Sache glauben notwendig Schritt letzte Tranche 15 Milliarde Euro bewilligen eigentlich Botschaft 15 Milliarde Euro letzt Tranche keineswegs samen Kapazität ESM Griechenland Aussicht stellen ausschöpfen ermöglichen möglicherweise Anspruch nehmen Anspruch nehmen letzte Tranche dienen Übergang Rückkehr Refinanzierung Kapitalmärkte begleiten Genauso Portugal Irland genauso Griechenland funktionieren Klar langen Programm Hilfe groß ergänzen Entscheidung Zukunft wichtig Laufzeitverlängerung Zinsstundung Krediterleichterungen schaffen Aussicht stellen Bundesminister Scholz Ehrhorn AfD gerne Zwischenfrage stellen Gestatten Bitte Minister vorgestern Frage konfrontieren ausfahren Bundestag August 2015 Rettungspaket zustimmen einzig wichtig Voraussetzung nämlich IWF Rettungspaket beteiligen Beteiligung geben rede explizit vorgestern Rettungspaket Wahrheit entsprechen vorgestern fragen wiederholen Frage Geschäftsgrundlage Zustimmung Bundestag erlöschen Auszahlung Rettungspaketes befürworten Dank schön Schönes Frage wiederholen gleichen Frage stellen wiederholen gleichen Antwort geben IWF dabeihaben IWF Kredit Höhe 10 Milliarde Euro laufen 2024 zurückzahlen Programm Rolle spielen IWF Geld kümmern vorstellen überlegen eigentlich Sinn obwohl ESM-Kreditrahmen ausschöpfen IWF 1,6 Milliarde obendrauf geben lassen gewissermaßen Situation wiederholen ausdrücklich IWF Nachprogrammüberwachung politische Zweck Beschluß Haus erreichen Übrigen Punkt kommen hinten Rede weglassen sagen verlängern Zahlungszeiträume helfen Kredittragfähigkeit sorgen Rückkehr Refinanzierung Kapitalmärkte gelingen einüben Praxis erreichen verzichten Zinsaufschlag Beschluß 2016 2017 ankündigen Gewinn Zentralbank anfangs geben bilateral Darlehen erhalten griechische Politik erreichbar Ding Weg Zustimmung bitten hören vereinbart sprechen kommen Nachprogrammüberwachung geben Anstrengung Bürgerin Bürger Griechenland Regierung unternehmen aufhören vergeblich gerne Politik Griechenland Zukunft entwickeln entsprechen Reformmaßnahmen letzt orientieren Klar Spielräume groß umso groß groß wirtschaftlich Erfolg Land Fall Nachprogrammüberwachung sicherstellen Griechenland Zukunft Reformpolitik verfolgen letzt einschlagen Schluss sagen Zeichen europäisch Solidarität geben erreichen europäisch Union gemeinsam zeigen zusammenhalten Lage Problem lösen Problem Wirtschaft organisieren Staatsfinanzen entwickeln ze

In [169]:
df['clean_text'] = df['clean_text'].apply(str.split).progress_apply(filtering)

  0%|          | 0/24647 [00:00<?, ?it/s]

In [164]:
df = txa_pd_read_sql('../resources/bt_reden_final.sqlite',"SELECT * FROM bt_reden_prepared_detailed")


DatabaseError: Execution failed on sql 'SELECT * FROM bt_reden_prepared_detailed': no such table: bt_reden_prepared_detailed

In [168]:
with open('../resources/stopwords_de.txt', encoding='utf-8') as file:
    stopwords = file.read().splitlines()


In [161]:
df.drop(columns=['full_text','lemmas'],axis=1,inplace=True)
df

speech_id                speaker         affiliation  \
0      ID194300100   Olaf Scholz            Bundesminister BMF   
1      ID194300200   Peter Boehringer       AfD                  
2      ID194300300   Eckhardt Rehberg       CDU/CSU              
3      ID194300400   Christian Dürr         FDP                  
4      ID194300500   Fabio De Masi          DIE LINKE            
...            ...             ...                ...            
24642  ID1916710300  Sonja Amalie Steffen   SPD                  
24643  ID1916710400  Friedrich Straetmanns  DIE LINKE            
24644  ID1916710500  Marco Bülow            Fraktionslos         
24645  ID1916710600  Michael Frieser        CDU/CSU              
24646  ID1916710700  Frank Schwabe          SPD                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [13]:
for index,speaker,aff in  zip(df.index,df['speaker'].values,df['affiliation'].values):
    if '90' in aff:
        df.loc[index,'affiliation']= "BÜNDNIS 90/DIE GRÜNEN"
        print(aff)

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
Bündnis 90/Die Grünen
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90/DIE GRÜNEN
BÜNDNIS 90

In [14]:
# database = '../../resources/bt_reden_final.sqlite'
# with sqlite3.connect(database) as con:
#     df.to_sql('bt_reden_prepared', con, index=False, if_exists='replace')
# con.close()


In [155]:
df.head(12)

speech_id                 speaker            affiliation  \
0   ID194300100  Olaf Scholz             Bundesminister BMF      
1   ID194300200  Peter Boehringer        AfD                     
2   ID194300300  Eckhardt Rehberg        CDU/CSU                 
3   ID194300400  Christian Dürr          FDP                     
4   ID194300500  Fabio De Masi           DIE LINKE               
5   ID194300600  Sven-Christian Kindler  BÜNDNIS 90/DIE GRÜNEN   
6   ID194300700  Peter Boehringer        AfD                     
7   ID194300800  Sven-Christian Kindler  BÜNDNIS 90/DIE GRÜNEN   
8   ID194300900  Achim Post              SPD                     
9   ID194301000  Harald Weyel            AfD                     
10  ID194301100  Christian Haase         CDU/CSU                 
11  ID194301200  Christian Dürr          FDP                     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           